In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
import numpy as np
from src.config import *

# Create ligand-receptor interaction matrix
 - read in ligand-receptor interactions queried from OmniPath
 - filter resource and LINCS data from overlapping ligands and receptors
 - create matrix and save

## Read in data

In [7]:
lr_interactions = pd.read_csv(LIG_REC_DF, index_col =0)

In [8]:
lr_interactions = lr_interactions[['source_genesymbol', 'target_genesymbol']]

In [9]:
sig_info = pd.read_table(f'{LINCS_DATA_DIR}siginfo_beta.txt', low_memory=False)
cols = ['sig_id', 'pert_type', 'pert_id', 'cmap_name', 'pert_idose', 'pert_itime', 'cell_iname']
sig_info = sig_info[cols]

In [10]:
lr_interactions.head(3)

,source_genesymbol,target_genesymbol
1,EPO,EPOR
2,CXCL16,CXCR6
3,KITLG,KIT


## Filter data

In [11]:
receptors = set(sig_info.cmap_name) & set(lr_interactions.target_genesymbol)
ligands = set(sig_info.cmap_name) & set(lr_interactions.source_genesymbol)

In [12]:
# review data source
receptors & ligands

set()

In [13]:
# filter based on the LINCS dataset
lr_interactions = lr_interactions[(lr_interactions['source_genesymbol'].isin(ligands)) & (lr_interactions['target_genesymbol'].isin(receptors))]
lr_interactions = lr_interactions.reset_index(drop = True)

## Create matrix

In [14]:
lincs_translate_receptor_ligands = lr_interactions[['source_genesymbol', 'target_genesymbol']].copy()
lincs_translate_receptor_ligands['sign'] = 1
lincs_translate_receptor_ligands.columns = ['ligand', 'receptor', 'sign']

In [15]:
table = pd.pivot_table(lincs_translate_receptor_ligands, values='sign', index='ligand',
                    columns='receptor', fill_value=0)

## Save matrix

In [16]:
table.to_csv(LIG_REC_MATRIX)